Neural Model Identification: \
Trains a neural network based on data from the physics model to use as the predictive model
Note: You need to install the matlab engine to be able to run matlab code from pythone

In [1]:
import matplotlib.pyplot as plt

In [3]:
import os
import pickle
import math
#from utils import kits
import time
import random 
import numpy as np
import tensorflow as tf
import scipy.io
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
#from skopt.utils import use_named_args
#from skopt import gp_minimize
#from skopt.space import Real, Categorical, Integer
from sklearn import preprocessing

In [4]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [5]:
import scipy.io
model_data = scipy.io.loadmat('Data.mat')

In [6]:
Zin=model_data.get("Zin")
Zout=model_data.get("Zout")

In [7]:
def swish(x, beta=1):
    return x * sigmoid(beta * x)

In [8]:
def get_model_layers(nodes, layers, activation, learning_rate, training=None):
    if activation == 'swish':
        activation = swish
    else:
        activation = activation

    inputs = tf.keras.layers.Input(shape=(in_dim,))
  #  xadd = (np.array(inputs[:, :out_dim])).reshape(None,1)
   # print(inputs[:, :out_dim].shape)
    #print(xadd.shape)
    x = tf.keras.layers.Dense(units=nodes, activation=activation)(inputs)
    for i in range(layers - 1):  # at least one layer which has the parameter input as input
        x = tf.keras.layers.Dense(units=nodes, activation=activation)(x)

    #x = tf.keras.layers.Dropout(rate=0.1)(x, training=training)
    outputs = tf.keras.layers.Dense(out_dim, activation="linear")(x)
    #outputs = tf.keras.layers.Add()([outputs, inputs[:, 2:]])

    #outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]])
    # outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]]) NN4UQ
    #    out = keras.layers.Dense(units=3, activation= "linear")(added)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    optimizer = optimizers.Adagrad(lr=learning_rate)# amsgrad=True)
    model.compile(optimizer=optimizer,
                  loss='mean_absolute_error')
    return model

In [9]:
get_custom_objects().update({'swish': Activation(swish)})

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(Zin, Zout, shuffle=True, train_size=0.8, random_state=42)
in_dim = X_train.shape[1]
out_dim = Y_train.shape[1]

In [11]:
learning_rate=0.05#gp_result.x[0]
num_layers = 4 #gp_result.x[1]
num_nodes = 6 #gp_result.x[2]
batch_size=10 #gp_result.x[3]
epochs=300
activation='swish'
DropModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                             activation=activation, learning_rate=learning_rate,
                             training=True)  # True-activates dropout during prediction
DropModel.summary()
DropModel.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data = (X_test, Y_test))

DetModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                            activation=activation, learning_rate=learning_rate)
#name = '{}_{}_{}_{}'.format(num_nodes, num_layers, activation, learning_rate)
#DropModel.save(root + 'results/nc/' + name)
DetModel.set_weights(DropModel.get_weights())
score = DetModel.evaluate(X_test, Y_test, verbose=0)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 36        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 21        
Total par

1440/1440 [==============================] - 0s 74us/sample - loss: 0.0032 - val_loss: 0.0040
Epoch 60/300
1440/1440 [==============================] - 0s 72us/sample - loss: 0.0031 - val_loss: 0.0034
Epoch 61/300
1440/1440 [==============================] - 0s 81us/sample - loss: 0.0031 - val_loss: 0.0024
Epoch 62/300
1440/1440 [==============================] - 0s 72us/sample - loss: 0.0030 - val_loss: 0.0028
Epoch 63/300
1440/1440 [==============================] - 0s 72us/sample - loss: 0.0030 - val_loss: 0.0027
Epoch 64/300
1440/1440 [==============================] - 0s 76us/sample - loss: 0.0029 - val_loss: 0.0044
Epoch 65/300
1440/1440 [==============================] - 0s 74us/sample - loss: 0.0027 - val_loss: 0.0026
Epoch 66/300
1440/1440 [==============================] - 0s 72us/sample - loss: 0.0026 - val_loss: 0.0037
Epoch 67/300
1440/1440 [==============================] - 0s 73us/sample - loss: 0.0026 - val_loss: 0.0023
Epoch 68/300
1440/1440 [==========================

1440/1440 [==============================] - 0s 124us/sample - loss: 0.0017 - val_loss: 0.0017
Epoch 136/300
1440/1440 [==============================] - 0s 127us/sample - loss: 0.0017 - val_loss: 0.0016
Epoch 137/300
1440/1440 [==============================] - 0s 124us/sample - loss: 0.0017 - val_loss: 0.0016
Epoch 138/300
1440/1440 [==============================] - 0s 126us/sample - loss: 0.0017 - val_loss: 0.0018
Epoch 139/300
1440/1440 [==============================] - 0s 123us/sample - loss: 0.0018 - val_loss: 0.0025
Epoch 140/300
1440/1440 [==============================] - 0s 123us/sample - loss: 0.0017 - val_loss: 0.0016
Epoch 141/300
1440/1440 [==============================] - 0s 123us/sample - loss: 0.0017 - val_loss: 0.0022
Epoch 142/300
1440/1440 [==============================] - 0s 123us/sample - loss: 0.0017 - val_loss: 0.0027
Epoch 143/300
1440/1440 [==============================] - 0s 126us/sample - loss: 0.0017 - val_loss: 0.0022
Epoch 144/300
1440/1440 [========

1440/1440 [==============================] - 0s 78us/sample - loss: 0.0015 - val_loss: 0.0015
Epoch 211/300
1440/1440 [==============================] - 0s 77us/sample - loss: 0.0015 - val_loss: 0.0016
Epoch 212/300
1440/1440 [==============================] - 0s 77us/sample - loss: 0.0015 - val_loss: 0.0017
Epoch 213/300
1440/1440 [==============================] - 0s 73us/sample - loss: 0.0015 - val_loss: 0.0019
Epoch 214/300
1440/1440 [==============================] - 0s 72us/sample - loss: 0.0015 - val_loss: 0.0017
Epoch 215/300
1440/1440 [==============================] - 0s 72us/sample - loss: 0.0015 - val_loss: 0.0016
Epoch 216/300
1440/1440 [==============================] - 0s 72us/sample - loss: 0.0014 - val_loss: 0.0018
Epoch 217/300
1440/1440 [==============================] - 0s 81us/sample - loss: 0.0015 - val_loss: 0.0016
Epoch 218/300
1440/1440 [==============================] - 0s 77us/sample - loss: 0.0015 - val_loss: 0.0015
Epoch 219/300
1440/1440 [=================

Epoch 286/300
1440/1440 [==============================] - 0s 71us/sample - loss: 0.0014 - val_loss: 0.0015
Epoch 287/300
1440/1440 [==============================] - 0s 71us/sample - loss: 0.0014 - val_loss: 0.0019
Epoch 288/300
1440/1440 [==============================] - 0s 71us/sample - loss: 0.0014 - val_loss: 0.0015
Epoch 289/300
1440/1440 [==============================] - 0s 70us/sample - loss: 0.0014 - val_loss: 0.0016
Epoch 290/300
1440/1440 [==============================] - 0s 70us/sample - loss: 0.0014 - val_loss: 0.0015
Epoch 291/300
1440/1440 [==============================] - 0s 71us/sample - loss: 0.0014 - val_loss: 0.0015
Epoch 292/300
1440/1440 [==============================] - 0s 73us/sample - loss: 0.0014 - val_loss: 0.0015
Epoch 293/300
1440/1440 [==============================] - 0s 71us/sample - loss: 0.0014 - val_loss: 0.0015
Epoch 294/300
1440/1440 [==============================] - 0s 71us/sample - loss: 0.0014 - val_loss: 0.0015
Epoch 295/300
1440/1440 [===

In [12]:
weights = DropModel.get_weights() # returs a numpy list of weights

In [13]:
Wmat = np.array(weights)

/Users/gmakr/opt/anaconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [14]:
def predictive_model_resnet(zk,Wmat): 
    #Assuming you can pass extra arguments to the function for casadi, otherwsise
    #Number of hidden layers and nodes are given once when we get the optimal resnet
    #The Bayesian Optimization Loop will be changing 
    n_hidden_layers = 4 
    n_nodes = 6
    model_input = zk.reshape(1,5)
    Wmat_shape = Wmat.shape[0]
    
    #Initialize from input 
    sum0 = np.matmul(zk,Wmat[0]) #Matrix multiplication weights*inputs
    sumbias0 = sum0 + Wmat[1] #Add bias to construct y = sum(wi*xi) + b
    #neuron_out = swish_eval(sum1bias)
    neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation, out = fact(y)
    #They are vector quantitites so you might need to change stuff a bit with symbolic vars
    print(neuron_out.shape)

    #Forward prediction in hidden layers
    for k in range(n_hidden_layers-1):
        sum0 = np.matmul(neuron_out,Wmat[2*k+2])
        sumbias0 = sum0 + Wmat[2*k+3]
        neuron_out = sumbias0/(1+np.exp(-sumbias0)) #explicitly writing swish activation

    #Final output layer with linear activation
    sum0 = np.matmul(neuron_out,Wmat[Wmat_shape-2])
    sumbias0 = sum0 + Wmat[Wmat_shape-1]
    neuron_out = sumbias0  #linear activation

    return neuron_out

In [15]:
zk=Zin[10,:].reshape(1,5)

In [16]:
zkplusone  = predictive_model_resnet(zk,Wmat)

(1, 6)


In [17]:
zk[0,0]

0.15476049490267696

In [18]:
print('Current State:', zk,'Future State', zkplusone)

Current State: [[0.15476049 0.33804598 0.12108554 0.00837934 0.01562051]] Future State [[0.11691863 0.00731835 0.01316322]]


In [44]:
#zkplusone_plant = eng.BCR_Discrete(matlab.double([[(zk[0,2]),(zk[0,3]),(zk[0,2])]]),matlab.double([zk[0,0]]),matlab.double([zk[0,1]]),82.3,1.0,0.9)